In [42]:
import re
from sec_api import ExtractorApi
from bs4 import BeautifulSoup
import os
from sec_edgar_downloader import Downloader
import requests
import pandas as pd

In [43]:
# API key
extractorApi = ExtractorApi("1fce8f9e2cbfca47996dc5fa148744e12cb637712e36759326aab5a3dafcfdfe")

In [44]:
ticker = "V"
headers = {"User-Agent": "gawegi8003@agafx.com"}

In [45]:
# to obtain the central index key for ticker

def cik_matching_ticker(ticker, headers=headers):
    ticker = ticker.upper().replace(".", "-")
    ticker_json = requests.get("https://www.sec.gov/files/company_tickers.json", headers=headers).json()

    for company in ticker_json.values():
        if company['ticker'] == ticker:
            cik = str(company['cik_str']).zfill(10)      #cik = str(company['cik_str']).zfill(10) , to fill to get 10 digit
            return cik
    raise ValueError(f'The ticker for {ticker} is not available in the SEC dataset')

cik = cik_matching_ticker(ticker, headers=headers)
cik

'0001403161'

In [46]:
# to get filing details
def get_submission_data_for_ticker(ticker, headers=headers):
    cik = cik_matching_ticker(ticker)
    url = f"https://data.sec.gov/submissions/CIK{cik}.json"
    company_json = requests.get(url, headers=headers).json()

    return pd.DataFrame(company_json["filings"]['recent'])

filings = get_submission_data_for_ticker(ticker, headers=headers)
filings

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
0,0001127602-24-013502,2024-04-26,2024-04-26,2024-04-26T19:55:58.000Z,,4,,,,4292,0,0,xslF345X05/form4.xml,PRIMARY DOCUMENT
1,0001965353-24-000153,2024-04-26,,2024-04-26T16:01:39.000Z,33,144,001-33977,24883400,,4786,0,0,xsl144X01/primary_doc.xml,
2,0001403161-24-000030,2024-04-24,2024-03-31,2024-04-23T18:06:59.000Z,34,10-Q,001-33977,24866218,,8734637,1,1,v-20240331.htm,10-Q
3,0001403161-24-000027,2024-04-23,2024-04-23,2024-04-23T16:05:39.000Z,34,8-K,001-33977,24864835,"2.02,8.01,9.01",1307816,1,1,v-20240423.htm,8-K
4,9999999995-24-000887,2024-04-08,,2024-04-09T00:15:24.000Z,33,EFFECT,333-276747,24831037,"S-4,,2024-04-08 09:00:00",1787,0,0,xslEFFECTX01/primary_doc.xml,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0001127602-12-022414,2012-07-17,2012-07-13,2012-07-17T16:36:18.000Z,,4,,,,11239,0,0,xslF345X03/form4.xml,PRIMARY DOCUMENT
996,0001193125-12-302336,2012-07-16,2012-07-13,2012-07-13T20:13:14.000Z,34,8-K,001-33977,12962587,"1.01,8.01,9.01",551163,0,0,d378086d8k.htm,FORM 8-K
997,0000000000-12-036275,2012-07-11,,2012-07-11T16:20:33.000Z,,UPLOAD,,,,36094,0,0,filename1.pdf,
998,0001127602-12-022062,2012-07-10,2012-07-09,2012-07-10T18:46:33.000Z,,4,,,,10236,0,0,xslF345X03/form4.xml,PRIMARY DOCUMENT


In [47]:
# Filtering 10K report
def get_filtered_filings(ticker, just_accession_numbers = False, headers=headers):
    company_filings_df = get_submission_data_for_ticker(ticker, headers=headers)
    df = company_filings_df[company_filings_df['form'] == '10-K']
    if just_accession_numbers:
        df = df.set_index('reportDate')
        accession_df = df['accessionNumber']
        return accession_df
    else:
        return df

filtered_report = get_filtered_filings(ticker, just_accession_numbers = False, headers = headers)
filtered_report

,accessionNumber,filingDate,reportDate,acceptanceDateTime,act,form,fileNumber,filmNumber,items,size,isXBRL,isInlineXBRL,primaryDocument,primaryDocDescription
70,0001403161-23-000099,2023-11-15,2023-09-30,2023-11-15T16:07:14.000Z,34,10-K,001-33977,231410612,,19718853,1,1,v-20230930.htm,10-K
182,0001403161-22-000081,2022-11-16,2022-09-30,2022-11-16T16:06:43.000Z,34,10-K,001-33977,221395232,,25116725,1,1,v-20220930.htm,10-K
273,0001403161-21-000060,2021-11-18,2021-09-30,2021-11-18T16:06:50.000Z,34,10-K,001-33977,211424468,,25113866,1,1,v-20210930.htm,10-K
365,0001403161-20-000070,2020-11-19,2020-09-30,2020-11-19T16:03:34.000Z,34,10-K,001-33977,201328832,,25094543,1,1,v-20200930.htm,10-K
449,0001403161-19-000050,2019-11-14,2019-09-30,2019-11-14T15:35:00.000Z,34,10-K,001-33977,191219544,,28748946,1,1,v093019form10k.htm,10-K
510,0001403161-18-000055,2018-11-16,2018-09-30,2018-11-16T16:10:27.000Z,34,10-K,001-33977,181189947,,18828853,1,0,v093018.htm,10-K
571,0001403161-17-000044,2017-11-17,2017-09-30,2017-11-16T20:07:25.000Z,34,10-K,001-33977,171209440,,21478842,1,0,v093017.htm,10-K
638,0001403161-16-000058,2016-11-15,2016-09-30,2016-11-15T16:44:02.000Z,34,10-K,001-33977,162000223,,20871018,1,0,v093016.htm,10-K
718,0001403161-15-000013,2015-11-20,2015-09-30,2015-11-19T18:24:51.000Z,34,10-K,001-33977,151244628,,17272826,1,0,v093015.htm,FORM 10-K
791,0001403161-14-000017,2014-11-21,2014-09-30,2014-11-20T18:57:50.000Z,34,10-K,001-33977,141240400,,22535629,1,0,v09301410-k.htm,10-K


In [48]:
# accession number dataframe
accession_df = filtered_report[["reportDate","accessionNumber"]].copy()
accession_df['accessionNumber'] = accession_df['accessionNumber'].str.replace('-', '')
accession_df['reportDate'] = accession_df['reportDate'].str.replace('-', '')
accession_df.rename(columns={'accessionNumber': 'Accession_Number', 'reportDate': 'Date'}, inplace=True)

# Reset the index of the new DataFrame
accession_df.reset_index(drop=True, inplace=True)
accession_df


,Date,Accession_Number
0,20230930,000140316123000099
1,20220930,000140316122000081
2,20210930,000140316121000060
3,20200930,000140316120000070
4,20190930,000140316119000050
5,20180930,000140316118000055
6,20170930,000140316117000044
7,20160930,000140316116000058
8,20150930,000140316115000013
9,20140930,000140316114000017


In [49]:
# for filing_url

cik = cik.lstrip('0')
ticker.lower()
filing_url_list = []

for i in range(len(accession_df['Date'])):
    filing_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_df['Accession_Number'][i]}/{ticker}-10k_{accession_df['Date'][i]}.htm"
    filing_url_list.append(str(filing_url))

filing_url_list

['https://www.sec.gov/Archives/edgar/data/1403161/000140316123000099/V-10k_20230930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316122000081/V-10k_20220930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316121000060/V-10k_20210930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316120000070/V-10k_20200930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316119000050/V-10k_20190930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316118000055/V-10k_20180930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316117000044/V-10k_20170930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316116000058/V-10k_20160930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316115000013/V-10k_20150930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316114000017/V-10k_20140930.htm',
 'https://www.sec.gov/Archives/edgar/data/1403161/000140316113000011/V-10k_20130930.htm',
 'https://

In [50]:
def cleaning_text(text):
    
    pattern = r"##TABLE_START.*?##TABLE_END"
    text = re.sub(pattern, "", text, flags=re.DOTALL)
    text = text.replace("&#160;", "")
    text = text.replace("&#8226;", "")
    return text

In [51]:
def getting_text_for_10k(filing_url, item_number):
    section = extractorApi.get_section(filing_url, item_number, 'text')
    item = cleaning_text(section)
    return item

In [52]:
# Using extractorApi to get data

path = f"D:/Fintech_lab/{ticker}"
os.mkdir(path)

for i in range(len(accession_df['Date'])):
    filing_url = f"https://www.sec.gov/Archives/edgar/data/{cik}/{accession_df['Accession_Number'][i]}/{ticker}-10k_{accession_df['Date'][i]}.htm"
    
    article_1 = getting_text_for_10k(filing_url, '1')
    article_1A = getting_text_for_10k(filing_url, '1A')
    article_5 = getting_text_for_10k(filing_url, '5')
    article_6 = getting_text_for_10k(filing_url, '6')
    article_7 = getting_text_for_10k(filing_url, '7')
    article_7A = getting_text_for_10k(filing_url, '7A')
    article_8 = getting_text_for_10k(filing_url, '8')

    report = article_1 + ' \n' + article_1A + ' \n' + article_5 + ' \n'  + article_6 + ' \n' + article_7 + '\n' + article_7A + ' \n' + article_8

    # # Storing text file
    file_path = f"D:/Fintech_lab/{ticker}/{accession_df['Date'][i]}.txt"
    with open(file_path, "w") as file:
        file.write(report)

    print("Text file created successfully at:", file_path)
    

Text file created successfully at: D:/Fintech_lab/V/20230930.txt
Text file created successfully at: D:/Fintech_lab/V/20220930.txt
Text file created successfully at: D:/Fintech_lab/V/20210930.txt
Text file created successfully at: D:/Fintech_lab/V/20200930.txt
Text file created successfully at: D:/Fintech_lab/V/20190930.txt
Text file created successfully at: D:/Fintech_lab/V/20180930.txt
Text file created successfully at: D:/Fintech_lab/V/20170930.txt
Text file created successfully at: D:/Fintech_lab/V/20160930.txt


Exception: API error: 429 - {"status":429,"error":"You send a lot of requests. We like that. But you exceeded the free query limit of 100 requests. Upgrade your account to get unlimited access. Visit sec-api.io for more."}